# Control-law design for full-state feedback

The purpose of the control law is to allow us to assign as set of poles locations for the close-loop system that will correspond to satisfactory dynamic response in term of rise time and other measures of transient response. 

## Finding the control law

How does it change the state equation when the system is inside a control loop? Below it's show the figure of a design block scheme:

|          | 
|:----------:|
|![control law\label{controllaw}](images/control-law.png)|

<p style="text-alignment=left;"><a name="fig8">Fig. 8</a> - Assumed system for control law design</p>

\begin{equation}
u = -\pmb{Kx} = \begin{bmatrix}K_1 & K_2 & ... K_n \end{bmatrix} \label{eq:K} 
\end{equation}

Equation $\eqref{eq:K}$ tell us that the system has a constant matrix in the state-vector feedback path gains, $K1,K2,....K_n$, and because there are n roots of the system, it's possible that there are enough degree of freedom to select arbitrarily and desired root location by choosing the proper value of $K_i$. Substituting the feedback law given in $\eqref{eq:K}$ into the system described by $\eqref{eq:stat-form}$ yields:  

\begin{equation}
\dot{\pmb{x}} = \pmb{Fx - GKx} = (\pmb{F - GK}) \pmb x
\end{equation}

Now writing:
\begin{equation}
\dot{\pmb x}(t) = p_i e^{p_i t} \pmb x_0 = (\pmb{F-GK}) \pmb x = (\pmb{F-GK}) e^{p_i t} \pmb x_0
\end{equation}
\begin{equation}
\pmb x(0) = \pmb x_0
\end{equation}

or

\begin{equation}
(\pmb{F-GK}) \pmb x_0 = p_i \pmb{x}_0
\end{equation}

This is an eigenvalues problem that it can be solved if and only if 

\begin{equation}
det[s \pmb I - (\pmb{F-GK})] = 0 \label{eq:det} 
\end{equation}

When evaluate, this yield an nth order polynomial in s containing the gains **K** so the roots of $\eqref{eq:det}$ are in the *desirable* locations. We assume that desired locations are known, say,

\begin{equation}s=s_1,s_2,....,s_n\end{equation}

Then the corresponding desired (control) characteristic equation is

\begin{equation}
\alpha_c(s) = (s-s_1)(s-s2)....(s-s_n) = 0 \label{eq:alpha} 
\end{equation}

Hence the required elements of **K** are obtained by matching coefficients $\eqref{eq:det}$ and $\eqref{eq:alpha}$ ,forcing the systems's characteristic equation to be identical to the desired characteristic equation and the closed loop poles at the desired locations.

Calculating the gains by using this technique becomes rather tedious when the order of the system is higher than 3. There are another technique that use the control canonical form to simplify the matrix and finishing with the match between the above equations. However there is an alternative to this method called **Ackerman's formula** divided in three steps process of converting to $(\pmb F_c \pmb G_c)$ solving for the gains, and converting back again into the very compact form:  

\begin{equation}
\pmb K = [0,0,.....0,1]{\mathcal{C}}^{-1} \alpha_c({\pmb F})
\end{equation}

where $\mathcal C$ is the controllability matrix in $\eqref{eq:C}$, n gives the order of the system and the number of the state variables and : 

\begin{equation}
\alpha_c (\pmb F) = \pmb F^n + \alpha_1 \pmb F^{n-1} + ..... + \alpha_n \pmb I
\end{equation}

where the $\alpha_i$ are the coefficient corresponding to the desired poles locations.

As an example it's simulated a control-law with different pole placement to show how the cart pendulum change the overshoot and robustness. 
```
clear all, close all, clc

m = 1;
M = 5;
L = 2;
g = -10;
d = 1;

s = 1; % pendulum up (s=1)

A = [0 1 0 0;
    0 -d/M -m*g/M 0;
    0 0 0 1;
    0 -s*d/(M*L) -s*(m+M)*g/(M*L) 0];

B = [0; 1/M; 0; s*1/(M*L)];
eig(A)

rank(ctrb(A,B))  % is it controllable

%%  Pole placement
% For more complex case a more reliable formula is available with 
% function place
% p is a vector of desired eigenvalues
p0 = [-.01; -.02; -.03; -.04]; % not enough
p1 = [-.3; -.4; -.5; -.6];  % just barely
p2 = [-1; -1.1; -1.2; -1.3]; % good
p3 = [-2; -2.1; -2.2; -2.3]; % aggressive
p4 = [-3; -3.1; -3.2; -3.3]; % aggressive
%p = [-3.5; -3.6; -3.7; -3.8]; % breaks
% create a list of vectors
list_p = {p1,p2,p3,p4};
% K = lqr(A,B,Q,R);

% Plot a list of non-minimum phase zero 
for k = 1:numel(list_p)
    tspan = 0:.001:30;
    vec_p = list_p{k};
    % Used place instead acker function
    K = place(A,B,vec_p)
    y0 = [-3; 0; pi+.1; 0];
    [t,y] = ode45(@(t,y)cartpend(y,m,M,L,g,d,-K*(y-[1; 0; pi; 0])),tspan,y0);    
    plot(t,y(:,1)); hold on;
end
```
Print out the K values:
```
K =

   -0.0360   -1.3420   71.9720   18.6840


K =

   -1.7160   -7.0260  142.5320   58.0520


K =

  -21.2520  -40.6460  379.6040  165.2920


K =

  -98.2080 -125.8660  851.5160  375.7320
```

|          |
|:----------:|
|![modalblock\label{modalblock}](images/poleplacement.png)|

<p style="text-alignment=left;"><a name="fig9">Fig. 9</a> - Some $K$ values based on different pole placement. The start position is x = -3 and $\theta= \pi +0.1$. Finale reference position is x = 1 $\theta = \pi$. It's worthwhile to note than more aggressively is equal to require more energy and less robustness </p>

It should be to note the system has to work harder to move pole long way (large gains) and reduce the control effort. In addition it's necessary to mention that the more aggressive will be my control the less robust the system will be because non linearity comes out respect the approximation done.

## How to select pole locations for a good design
As shown before, when selecting pole location it is always useful to keep in mind that control effort required is related to how far the open loop poles are moved by the feedback. Furthermore, when a zero is near a pole, the system may be nearly uncontrollable. The designer philosophy will take in account to fix only the undesirable aspects of the open loop response and avoids either large increases bandwidth or efforts to move poles that are near zeros will typically allow smaller gains, and thus smaller controls actuators.
There are 2 main ways to overcome the problem. First deals dominant second order poles, second is Linear Quadratic Regulator; in this paper it will be show the second because more suited with state variables in feedback controls for higher dimension.

### Linear quadratic regulator

We have seen in the previous sections that if $(F, G)$ is controllable, then it is possible to arbitrarily manipulate the eigenvalues of the closed-loop system $(\pmb{A − BK})$ through choice of a full-state feedback control law $u = −\pmb{Kx}$. This implicitly assumes that full-state measurements are available (i.e., $H = I$ and $J = 0$, so that $\pmb y = \pmb x$). Given a controllable system, and either measurements of the full-state or an observable system with a full-state estimate, there are many choices of stabilizing control laws $u = −\pmb{Kx}$. It is possible to make the eigenvalues of the closed-loop system ($\pmb A − \pmb{BK})$ arbitrarily stable, placing them as far as desired in the left-half of the complex plane. However, overly stable eigenvalues may require exceedingly expensive control expenditure and might also result in actuation signals that exceed maximum allowable values. Choosing very stable eigenvalues may also cause the control system to over-react to noise and disturbances, much as a new driver will over-react to vibrations in the steering wheel, causing the closed-loop system to jitter. Over stabilization can counter-intuitively degrade robustness and may lead to instability if there are small time delays or unmodeled dynamics. Choosing the best gain matrix $\pmb K$ to stabilize the system without expending too much control effort is an important goal in optimal control. A balance must be struck between the stability of the closed-loop system and the aggressiveness of control. It is important to take control expenditure into account:
 1. to prevent the controller from over-reacting to high-frequency noise and disturbances
 2. that actuation does not exceed maximum allowed amplitudes
 3. that control is not prohibitively expensive. 

In particular, the cost function

\begin{equation}
J(t) = \int\limits_{0}^{t} {\pmb x(\tau)^* \pmb Q x(\tau) + \pmb u(\tau)^* \pmb R u(\tau) d\tau}
\label{eq:costfunc} 
\end{equation}

balances the cost of effective regulation of the state with the cost of control. The matrices **Q** and **R** weight the cost of deviations of the state from zero and the cost of actuation, respectively. The matrix **Q** is positive semi-definite, and **R** is positive definite; these matrices are often diagonal, and the diagonal elements may be tuned to change the relative importance of the control objectives.
Adding such a cost function makes choosing the control law a well-posed optimization problem. The linear-quadratic-regulator (LQR) control law $u = −\pmb K_r \pmb x$ is designed to minimize $J = \lim\limits_{t \to \infty}{J(t)}$. LQR is so-named because it is a linear control law, designed for a linear system, minimizing a quadratic cost function, that regulates the state of the system to $J = \lim\limits_{t \to \infty}{\pmb x(t) = \pmb 0}$. Because the cost-function in $\eqref{eq:costfunc}$ is quadratic, there is an analytical solution for the optimal controller gains $K_r$ , given by

\begin{equation}
    \pmb K_r = \pmb R^{-1} \pmb G^* \pmb X
\end{equation}

where $\pmb X$ is the solution to an algebraic Riccati equation:

\begin{equation}
    \pmb{F^*X + XF - XGR^{-1}G^*X + Q}
\end{equation}

Solving the above Riccati equation for $\pmb X$, and hence for $\pmb K_r$ , is numerically robust and already implemented in many programming languages. In Matlab, $K_r$ is obtained via :
```
>> Kr = lqr(F,G,Q,R);
```

Here picture LQR

|           |
|-----------|
|![fig1.4](images/LQR.png)|

<p><a name="LQR"> Fig. 10 </a>- Solution for Linear Quadratic Regulator; full state system</p>